In [1]:
from azureml.core import Workspace, Model

# Connect to the Workspace

In [6]:
# Load the workspace from the saved config file
ws = Workspace.from_config()

## Deploy the model for inference

In [23]:
model = ws.models['quiddler_model']
print(model.name, 'version', model.version)

quiddler_model version 1


In [ ]:
model

In [25]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_pip_package("azureml-defaults")
myenv.add_pip_package("icevision[all]==0.1.4")
myenv.add_pip_package("inference-schema")
myenv.save('quiddler/inference-env.yml')

'quiddler/inference-env.yml'

In [26]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig

In [ ]:
# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = 'quiddler',
                                   entry_script="score.py",
                                   conda_file="inference-env.yml")

deployment_config = LocalWebservice.deploy_configuration()#(cpu_cores = 1, memory_gb = 1)

service_name = "quiddler-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

## Call the inference service

In [35]:
service.reload(wait=True)

Container has been successfully cleaned up.
Starting Docker container...
Docker container running.
Checking container health...
Local webservice is running at http://localhost:32773


In [36]:
endpoint = service.scoring_uri
print(endpoint)

http://localhost:32773/score


In [54]:
type(service)

azureml.core.webservice.local.LocalWebservice

In [51]:
# RAW DATA
import requests
# Load image data
data = open('/home/jeremy/Documents/data/quiddler/test/IMG_4904.jpg', 'rb').read()
# Post raw data to scoring URI
res = requests.post(url=endpoint, data=data, headers={'Content-Type': 'application/> octet-stream'})
res.content

b'"r,e,s,z,v,m"'

In [40]:
# JSON DATA
import base64
import requests
import json 

def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

b64img = get_base64_encoded_image('/home/jeremy/Documents/data/quiddler/test/IMG_4907.jpg')
input_json = json.dumps({"n_cards": 11, "hand": b64img})
headers = { 'Content-Type':'application/json' }
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.json())

g,z,v,th,m,qu,u,o,y,j,k


In [ ]:
print(service.get_logs())

In [30]:
print(service.swagger_uri)

http://localhost:32771/swagger.json


## Shut it down

In [64]:
service.delete()
print("Service deleted.")

Container has been successfully cleaned up.
Service deleted.


In [65]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


# Deploy into AKS

In [13]:
# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = 'quiddler',
                                   entry_script="score.py",
                                   conda_file="inference-env.yml")
service_name = "quiddler-service2"

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.compute import AksCompute

aks_target = AksCompute(ws,"jb-inf2")
# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

In [ ]:
primary, secondary = service.get_keys()
print(primary)

In [ ]:
service.scoring_uri

In [18]:
import requests
import json
import base64

# URL for the web service
endpoint = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

b64img = get_base64_encoded_image('/home/jeremy/Documents/data/quiddler/test/IMG_4907.jpg')
input_json = json.dumps({"n_cards": 11, "hand": b64img})
headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.json())

g,z,v,th,m,qu,u,o,y,j,k


In [19]:
service.delete()